# RAG (Retrieval-Augmented Generation) బేసిక్స్

ఈ ట్యుటోరియల్‌లో, మనం LangChain లో RAG (Retrieval-Augmented Generation) గురించి నేర్చుకుంటాము. RAG అనేది LLM ని మన స్వంత డేటాతో ఎన్‌హాన్స్ చేయడానికి ఉపయోగపడే టెక్నిక్.

## సెటప్

మొదట, మనం అవసరమైన లైబ్రరీలను ఇన్‌స్టాల్ చేసుకుందాం:

In [ ]:
# అవసరమైన లైబ్రరీలను ఇన్‌స్టాల్ చేయడం
!pip install langchain langchain-openai langchain-community faiss-cpu tiktoken

ఇప్పుడు, మనం OpenAI API కీని సెట్ చేద్దాం:

In [ ]:
import os
from dotenv import load_dotenv

# .env ఫైల్ నుండి API కీని లోడ్ చేయడం
load_dotenv()

# లేదా డైరెక్ట్‌గా సెట్ చేయడం (డెవలప్‌మెంట్ కోసం మాత్రమే, ప్రొడక్షన్‌లో ఉపయోగించవద్దు)
# os.environ["OPENAI_API_KEY"] = "మీ-API-కీ-ఇక్కడ-పెట్టండి"

## RAG అంటే ఏమిటి?

RAG (Retrieval-Augmented Generation) అనేది LLM ని మన స్వంత డేటాతో ఎన్‌హాన్స్ చేయడానికి ఉపయోగపడే టెక్నిక్. ఇది రెండు ప్రధాన స్టెప్స్‌ని కలిగి ఉంటుంది:

1. **Retrieval (రిట్రీవల్)**: ప్రశ్నకు సంబంధించిన డాక్యుమెంట్స్ లేదా ఇన్ఫర్మేషన్‌ని వెతకడం
2. **Generation (జనరేషన్)**: రిట్రీవ్ చేసిన ఇన్ఫర్మేషన్‌ని ఉపయోగించి, LLM ద్వారా సమాధానాన్ని జనరేట్ చేయడం

RAG ఎందుకు ముఖ్యమైనది?
1. **అప్-టు-డేట్ ఇన్ఫర్మేషన్**: LLM ట్రైనింగ్ డేటా కంటే తాజా సమాచారాన్ని అందించడం
2. **డొమైన్-స్పెసిఫిక్ నాలెడ్జ్**: LLM కి లేని స్పెషలైజ్డ్ నాలెడ్జ్‌ని అందించడం
3. **సోర్సింగ్ మరియు ట్రేసబిలిటీ**: జనరేట్ చేసిన కంటెంట్‌కి సోర్స్‌ని ప్రొవైడ్ చేయడం
4. **హాలుసినేషన్స్ తగ్గించడం**: LLM ఊహించిన సమాధానాలు ఇవ్వకుండా ఉండటం

## RAG పైపైలైన్

ఒక సాధారణ RAG పైపైలైన్‌లో ఈ క్రింది స్టెప్స్ ఉంటాయి:

1. **డాక్యుమెంట్ లోడింగ్**: డాక్యుమెంట్స్‌ని లోడ్ చేయడం (PDF, వెబ్‌పేజీలు, టెక్స్ట్ ఫైల్స్, మొదలైనవి)
2. **డాక్యుమెంట్ ప్రాసెసింగ్**: డాక్యుమెంట్స్‌ని చిన్న చిన్న చంక్స్‌గా విభజించడం
3. **ఎంబెడింగ్స్ జనరేషన్**: చంక్స్‌ని వెక్టర్ రిప్రజెంటేషన్స్‌గా మార్చడం
4. **వెక్టర్ స్టోర్**: ఎంబెడింగ్స్‌ని వెక్టర్ డేటాబేస్‌లో స్టోర్ చేయడం
5. **క్వెరీ ప్రాసెసింగ్**: యూజర్ ప్రశ్నని ఎంబెడింగ్‌గా మార్చడం
6. **రిట్రీవల్**: ప్రశ్నకు సంబంధించిన చంక్స్‌ని వెక్టర్ డేటాబేస్ నుండి రిట్రీవ్ చేయడం
7. **జనరేషన్**: రిట్రీవ్ చేసిన చంక్స్‌ని ఉపయోగించి, LLM ద్వారా సమాధానాన్ని జనరేట్ చేయడం

## సింపుల్ RAG ఎగ్జాంపుల్

ఇప్పుడు, మనం ఒక సింపుల్ RAG ఎగ్జాంపుల్‌ని చూద్దాం. ఈ ఎగ్జాంపుల్‌లో, మనం తెలుగు సినిమాల గురించి ఒక చిన్న డాక్యుమెంట్‌ని క్రియేట్ చేసి, దానిపై RAG ని అప్లై చేద్దాం.

In [ ]:
# అవసరమైన ఇంపోర్ట్స్
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import os

# తెలుగు సినిమాల గురించి డాక్యుమెంట్‌ని క్రియేట్ చేయడం
telugu_cinema_doc = """
# తెలుగు సినిమా చరిత్ర

తెలుగు సినిమా చరిత్ర 1912లో ప్రారంభమైంది, కానీ మొదటి తెలుగు టాకీ "భక్త ప్రహ్లాద" 1931లో విడుదలైంది. ఇది హెచ్.ఎం. రెడ్డి దర్శకత్వంలో తెరకెక్కింది.

## ప్రముఖ దర్శకులు

1. **కె. విశ్వనాథ్**: "శంకరాభరణం", "సాగర సంగమం", "స్వాతి ముత్యం" వంటి క్లాసిక్ చిత్రాలను తెరకెక్కించారు.
2. **బాపు**: "మువ్వలు", "మిస్టర్ పెళ్లాం", "రాధా గోపాలం" వంటి చిత్రాలతో తెలుగు సినిమాకు కళాత్మక విలువను జోడించారు.
3. **రాఘవేంద్ర రావు**: "అనురాగం", "సీతారామయ్య గారి మనవరాలు", "ప్రేమించుకుందాం రా" వంటి హిట్ చిత్రాలను తెరకెక్కించారు.
4. **ఎస్.ఎస్. రాజమౌళి**: "బాహుబలి" సిరీస్, "ఈగ", "మగధీర", "ఆర్ఆర్ఆర్" వంటి బ్లాక్‌బస్టర్ చిత్రాలతో అంతర్జాతీయ గుర్తింపు పొందారు.
5. **త్రివిక్రమ్ శ్రీనివాస్**: "అత్తారింటికి దారేది", "జులాయి", "అ.. అా" వంటి హిట్ చిత్రాలను తెరకెక్కించారు.

## ప్రముఖ నటులు

1. **ఎన్టీఆర్ (నందమూరి తారక రామారావు)**: తెలుగు సినిమా లెజెండ్, పౌరాణిక పాత్రలతో ప్రసిద్ధి చెందారు.
2. **ఏఎన్నార్ (అక్కినేని నాగేశ్వర రావు)**: 75 సంవత్సరాల కెరీర్‌తో 250కి పైగా చిత్రాలలో నటించారు.
3. **చిరంజీవి**: 80ల నుండి తెలుగు సినిమాలో టాప్ హీరోగా ఉన్నారు, 150కి పైగా చిత్రాలలో నటించారు.
4. **మహేష్ బాబు**: "ఒక్కడు", "పోకిరి", "స్పైడర్" వంటి హిట్ చిత్రాలతో ప్రసిద్ధి చెందారు.
5. **ప్రభాస్**: "బాహుబలి" సిరీస్‌తో అంతర్జాతీయ స్థాయిలో గుర్తింపు పొందారు.
6. **అల్లు అర్జున్**: "పుష్ప" చిత్రంతో పాన్-ఇండియా స్టార్‌గా మారారు.
7. **జూనియర్ ఎన్టీఆర్**: "ఆర్ఆర్ఆర్" చిత్రంతో గోల్డెన్ గ్లోబ్ అవార్డు గెలుచుకున్న తొలి భారతీయ చిత్రంలో నటించారు.
8. **రామ్ చరణ్**: "ఆర్ఆర్ఆర్" చిత్రంతో అంతర్జాతీయ గుర్తింపు పొందారు.

## ప్రముఖ నటీమణులు

1. **సావిత్రి**: "మాయాబజార్", "దేవదాసు" వంటి క్లాసిక్ చిత్రాలతో తెలుగు సినిమా గోల్డెన్ ఏజ్‌లో ప్రముఖ నటిగా ఉన్నారు.
2. **విజయశాంతి**: 80ల మరియు 90లలో లేడీ సూపర్‌స్టార్‌గా ప్రసిద్ధి చెందారు.
3. **అనుష్క శెట్టి**: "అరుంధతి", "బాహుబలి" వంటి బ్లాక్‌బస్టర్ చిత్రాలతో ప్రసిద్ధి చెందారు.
4. **సమంత**: "ఈగ", "అత్తారింటికి దారేది" వంటి హిట్ చిత్రాలతో ప్రసిద్ధి చెందారు.
5. **కీర్తి సురేష్**: "మహానటి" చిత్రంలో సావిత్రి పాత్రతో జాతీయ అవార్డు గెలుచుకున్నారు.

## ప్రముఖ చిత్రాలు

1. **మాయాబజార్ (1957)**: కాదంబరి, ఎన్టీఆర్, ఏఎన్నార్, ఎస్వీఆర్ నటించిన పౌరాణిక చిత్రం, తెలుగు సినిమా చరిత్రలో మైలురాయి.
2. **శంకరాభరణం (1979)**: కె. విశ్వనాథ్ దర్శకత్వంలో తెరకెక్కిన ఈ చిత్రం శాస్త్రీయ సంగీతాన్ని ప్రధాన థీమ్‌గా చేసుకుంది.
3. **బాహుబలి: ది బిగినింగ్ (2015)** మరియు **బాహుబలి: ది కన్‌క్లూజన్ (2017)**: ఎస్.ఎస్. రాజమౌళి దర్శకత్వంలో తెరకెక్కిన ఈ చిత్రాలు భారతీయ సినిమా చరిత్రలో అత్యధిక వసూళ్లు సాధించాయి.
4. **ఆర్ఆర్ఆర్ (2022)**: ఎస్.ఎస్. రాజమౌళి దర్శకత్వంలో, జూనియర్ ఎన్టీఆర్ మరియు రామ్ చరణ్ నటించిన ఈ చిత్రం ఆస్కార్ అవార్డు గెలుచుకుంది.
5. **పుష్ప: ది రైజ్ (2021)**: సుకుమార్ దర్శకత్వంలో, అల్లు అర్జున్ నటించిన ఈ చిత్రం పాన్-ఇండియా హిట్‌గా నిలిచింది.

## తెలుగు సినిమా ప్రస్తుత ట్రెండ్స్

1. **పాన్-ఇండియా చిత్రాలు**: ఇటీవలి సంవత్సరాలలో, తెలుగు సినిమాలు భారతదేశం అంతటా మరియు అంతర్జాతీయంగా విడుదల అవుతున్నాయి.
2. **బయోపిక్స్**: "మహానటి" (సావిత్రి జీవిత చరిత్ర), "ఎన్టీఆర్ కథానాయకుడు" (ఎన్టీఆర్ జీవిత చరిత్ర) వంటి బయోపిక్స్ ప్రజాదరణ పొందాయి.
3. **పీరియడ్ డ్రామాలు**: "బాహుబలి", "ఆర్ఆర్ఆర్" వంటి చారిత్రక మరియు పీరియడ్ డ్రామాలు ప్రేక్షకులను ఆకర్షిస్తున్నాయి.
4. **OTT ప్లాట్‌ఫామ్స్**: నెట్‌ఫ్లిక్స్, అమెజాన్ ప్రైమ్ వంటి OTT ప్లాట్‌ఫామ్స్ తెలుగు కంటెంట్‌ని ప్రపంచవ్యాప్తంగా అందిస్తున్నాయి.
"""

# డాక్యుమెంట్‌ని చంక్స్‌గా విభజించడం
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_text(telugu_cinema_doc)

# ఎంబెడింగ్స్‌ని క్రియేట్ చేయడం
embeddings = OpenAIEmbeddings()

# వెక్టర్ స్టోర్‌ని క్రియేట్ చేయడం
vectorstore = FAISS.from_texts(chunks, embeddings)

# రిట్రీవర్‌ని క్రియేట్ చేయడం
retriever = vectorstore.as_retriever()

# ప్రాంప్ట్‌ని క్రియేట్ చేయడం
template = """
మీరు తెలుగు సినిమాల గురించి నిపుణుడు. ఈ క్రింది కాంటెక్స్ట్ ఇన్ఫర్మేషన్‌ని ఉపయోగించి, ప్రశ్నకు సమాధానం ఇవ్వండి.
సమాధానం కాంటెక్స్ట్‌లో ఉన్న సమాచారానికి మాత్రమే పరిమితం చేయండి. మీకు తెలియని విషయాలు అడిగితే, "క్షమించండి, నాకు ఆ సమాచారం తెలియదు" అని చెప్పండి.

కాంటెక్స్ట్:
{context}

ప్రశ్న: {question}

సమాధానం:
"""

prompt = ChatPromptTemplate.from_template(template)

# LLM ని క్రియేట్ చేయడం
llm = ChatOpenAI(model="gpt-3.5-turbo")

# RAG చెయిన్‌ని క్రియేట్ చేయడం
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# RAG చెయిన్‌ని టెస్ట్ చేయడం
questions = [
    "తెలుగు సినిమా చరిత్రలో మొదటి టాకీ ఏది?",
    "ఎస్.ఎస్. రాజమౌళి తెరకెక్కించిన ప్రముఖ చిత్రాలు ఏవి?",
    "ఆస్కార్ అవార్డు గెలుచుకున్న తెలుగు చిత్రం ఏది?",
    "ప్రస్తుతం తెలుగు సినిమాలో ఉన్న ట్రెండ్స్ ఏమిటి?",
    "IPL లో సన్‌రైజర్స్ హైదరాబాద్ జట్టు గురించి చెప్పండి"
]

for question in questions:
    print(f"\nప్రశ్న: {question}")
    answer = rag_chain.invoke(question)
    print(f"సమాధానం: {answer}")

## IPL క్రికెట్ డేటాతో RAG ఎగ్జాంపుల్

ఇప్పుడు, మనం IPL క్రికెట్ గురించి ఒక డాక్యుమెంట్‌ని క్రియేట్ చేసి, దానిపై RAG ని అప్లై చేద్దాం.

In [ ]:
# IPL క్రికెట్ గురించి డాక్యుమెంట్‌ని క్రియేట్ చేయడం
ipl_cricket_doc = """
# ఇండియన్ ప్రీమియర్ లీగ్ (IPL) చరిత్ర మరియు సమాచారం

ఇండియన్ ప్రీమియర్ లీగ్ (IPL) అనేది భారతదేశంలో జరిగే ప్రొఫెషనల్ ట్వెంటీ20 క్రికెట్ లీగ్. ఇది 2008లో బీసీసీఐ (BCCI) ద్వారా స్థాపించబడింది. ప్రతి సంవత్సరం, ఎనిమిది నుండి పది జట్లు IPL ట్రోఫీ కోసం పోటీపడతాయి.

## IPL జట్లు

### చెన్నై సూపర్ కింగ్స్ (CSK)
- **స్థాపించబడింది**: 2008
- **హోమ్ గ్రౌండ్**: చెపాక్ స్టేడియం, చెన్నై
- **కెప్టెన్**: ఎం.ఎస్. ధోని
- **చాంపియన్షిప్స్**: 2010, 2011, 2018, 2021, 2023
- **ప్రముఖ ఆటగాళ్లు**: ఎం.ఎస్. ధోని, సురేష్ రైనా, రవీంద్ర జడేజా, డ్వేన్ బ్రావో

### ముంబై ఇండియన్స్ (MI)
- **స్థాపించబడింది**: 2008
- **హోమ్ గ్రౌండ్**: వాంఖడే స్టేడియం, ముంబై
- **కెప్టెన్**: రోహిత్ శర్మ
- **చాంపియన్షిప్స్**: 2013, 2015, 2017, 2019, 2020
- **ప్రముఖ ఆటగాళ్లు**: రోహిత్ శర్మ, జస్ప్రీత్ బుమ్రా, కీరన్ పొలార్డ్, సూర్యకుమార్ యాదవ్

### కోల్కతా నైట్ రైడర్స్ (KKR)
- **స్థాపించబడింది**: 2008
- **హోమ్ గ్రౌండ్**: ఈడెన్ గార్డెన్స్, కోల్కతా
- **కెప్టెన్**: శ్రేయస్ అయ్యర్
- **చాంపియన్షిప్స్**: 2012, 2014
- **ప్రముఖ ఆటగాళ్లు**: గౌతమ్ గంభీర్, సునీల్ నరైన్, ఆండ్రే రసెల్, వెంకటేష్ అయ్యర్

### సన్‌రైజర్స్ హైదరాబాద్ (SRH)
- **స్థాపించబడింది**: 2013 (డెక్కన్ చార్జర్స్ స్థానంలో)
- **హోమ్ గ్రౌండ్**: రాజీవ్ గాంధీ ఇంటర్నేషనల్ క్రికెట్ స్టేడియం, హైదరాబాద్
- **కెప్టెన్**: పాట్ కమిన్స్
- **చాంపియన్షిప్స్**: 2016
- **ప్రముఖ ఆటగాళ్లు**: డేవిడ్ వార్నర్, కేన్ విలియమ్సన్, భువనేశ్వర్ కుమార్, రషీద్ ఖాన్, హెన్రిచ్ క్లాసెన్
- **ప్రముఖ తెలుగు ఆటగాళ్లు**: టి. నటరాజన్

### రాయల్ చాలెంజర్స్ బెంగళూరు (RCB)
- **స్థాపించబడింది**: 2008
- **హోమ్ గ్రౌండ్**: ఎం. చిన్నస్వామి స్టేడియం, బెంగళూరు
- **కెప్టెన్**: ఫాఫ్ డు ప్లెసిస్
- **చాంపియన్షిప్స్**: ఇంకా లేవు
- **ప్రముఖ ఆటగాళ్లు**: విరాట్ కోహ్లి, ఏబీ డివిలియర్స్, క్రిస్ గేల్, గ్లెన్ మాక్స్‌వెల్

### రాజస్థాన్ రాయల్స్ (RR)
- **స్థాపించబడింది**: 2008
- **హోమ్ గ్రౌండ్**: సవాయి మాన్సింగ్ స్టేడియం, జైపూర్
- **కెప్టెన్**: సంజూ శాంసన్
- **చాంపియన్షిప్స్**: 2008
- **ప్రముఖ ఆటగాళ్లు**: షేన్ వార్న్, రాహుల్ ద్రావిడ్, షేన్ వాట్సన్, జోస్ బట్లర్

### పంజాబ్ కింగ్స్ (PBKS)
- **స్థాపించబడింది**: 2008
- **హోమ్ గ్రౌండ్**: పంజాబ్ క్రికెట్ అసోసియేషన్ ఐఎస్ బిందా స్టేడియం, మొహాలి
- **కెప్టెన్**: శిఖర్ ధావన్
- **చాంపియన్షిప్స్**: ఇంకా లేవు
- **ప్రముఖ ఆటగాళ్లు**: యువరాజ్ సింగ్, కేఎల్ రాహుల్, క్రిస్ గేల్, మహ్మద్ షమీ

### ఢిల్లీ క్యాపిటల్స్ (DC)
- **స్థాపించబడింది**: 2008 (ఢిల్లీ డేర్‌డెవిల్స్‌గా)
- **హోమ్ గ్రౌండ్**: అరుణ్ జైట్లీ స్టేడియం, ఢిల్లీ
- **కెప్టెన్**: రిషభ్ పంత్
- **చాంపియన్షిప్స్**: ఇంకా లేవు
- **ప్రముఖ ఆటగాళ్లు**: రిషభ్ పంత్, శిఖర్ ధావన్, కగిసో రబాడా, ప్రిథ్వీ షా

### గుజరాత్ టైటాన్స్ (GT)
- **స్థాపించబడింది**: 2022
- **హోమ్ గ్రౌండ్**: నరేంద్ర మోదీ స్టేడియం, అహ్మదాబాద్
- **కెప్టెన్**: హార్దిక్ పాండ్యా
- **చాంపియన్షిప్స్**: 2022
- **ప్రముఖ ఆటగాళ్లు**: హార్దిక్ పాండ్యా, రషీద్ ఖాన్, మహ్మద్ షమీ, శుభ్‌మన్ గిల్

### లక్నో సూపర్ జెయింట్స్ (LSG)
- **స్థాపించబడింది**: 2022
- **హోమ్ గ్రౌండ్**: భారత రత్న శ్రీ అటల్ బిహారీ వాజపేయి ఎకనా క్రికెట్ స్టేడియం, లక్నో
- **కెప్టెన్**: కేఎల్ రాహుల్
- **చాంపియన్షిప్స్**: ఇంకా లేవు
- **ప్రముఖ ఆటగాళ్లు**: కేఎల్ రాహుల్, క్విన్టన్ డి కాక్, మార్కస్ స్టోయినిస్, నికోలస్ పూరన్

## IPL విజేతలు

- **2008**: రాజస్థాన్ రాయల్స్
- **2009**: డెక్కన్ చార్జర్స్
- **2010**: చెన్నై సూపర్ కింగ్స్
- **2011**: చెన్నై సూపర్ కింగ్స్
- **2012**: కోల్కతా నైట్ రైడర్స్
- **2013**: ముంబై ఇండియన్స్
- **2014**: కోల్కతా నైట్ రైడర్స్
- **2015**: ముంబై ఇండియన్స్
- **2016**: సన్‌రైజర్స్ హైదరాబాద్
- **2017**: ముంబై ఇండియన్స్
- **2018**: చెన్నై సూపర్ కింగ్స్
- **2019**: ముంబై ఇండియన్స్
- **2020**: ముంబై ఇండియన్స్
- **2021**: చెన్నై సూపర్ కింగ్స్
- **2022**: గుజరాత్ టైటాన్స్
- **2023**: చెన్నై సూపర్ కింగ్స్

## సన్‌రైజర్స్ హైదరాబాద్ గురించి మరింత సమాచారం

సన్‌రైజర్స్ హైదరాబాద్ (SRH) అనేది హైదరాబాద్ నుండి వచ్చిన IPL ఫ్రాంచైజీ. ఇది 2013లో డెక్కన్ చార్జర్స్ స్థానంలో IPLలో చేరింది. సన్‌రైజర్స్ హైదరాబాద్ 2016లో IPL ట్రోఫీని గెలుచుకుంది, డేవిడ్ వార్నర్ కెప్టెన్‌గా ఉన్నప్పుడు.

### సన్‌రైజర్స్ హైదరాబాద్ ప్రముఖ విజయాలు

- **2016**: IPL చాంపియన్
- **2018**: IPL రన్నర్-అప్
- **2020**: IPL ప్లేఆఫ్స్ (3వ స్థానం)

### సన్‌రైజర్స్ హైదరాబాద్ ప్రముఖ ఆటగాళ్లు

1. **డేవిడ్ వార్నర్**: SRH కోసం అత్యధిక పరుగులు చేసిన ఆటగాడు, 2016 IPL విజయంలో కీలక పాత్ర పోషించాడు.
2. **భువనేశ్వర్ కుమార్**: SRH కోసం అత్యధిక వికెట్లు తీసిన ఆటగాడు, రెండుసార్లు పర్పుల్ క్యాప్ గెలుచుకున్నాడు (2016, 2017).
3. **రషీద్ ఖాన్**: ప్రపంచంలోని అగ్రశ్రేణి స్పిన్నర్లలో ఒకరు, SRH బౌలింగ్ విభాగంలో కీలక ఆటగాడు.
4. **కేన్ విలియమ్సన్**: న్యూజిలాండ్ కెప్టెన్, 2018లో SRHని ఫైనల్‌కు తీసుకెళ్లాడు.
5. **టి. నటరాజన్**: తమిళనాడు నుండి వచ్చిన పేస్ బౌలర్, 2020 సీజన్‌లో తన యార్కర్లతో ప్రసిద్ధి చెందాడు.

### సన్‌రైజర్స్ హైదరాబాద్ 2024 స్క్వాడ్

- **కెప్టెన్**: పాట్ కమిన్స్
- **బ్యాట్స్‌మెన్**: అభిషేక్ శర్మ, రాహుల్ త్రిపాఠి, గ్లెన్ ఫిలిప్స్, హెన్రిచ్ క్లాసెన్, ఆదర్శ్ సింగ్, అన్మోల్ప్రీత్ సింగ్, మయాంక్ అగర్వాల్, ట్రావిస్ హెడ్
- **ఆల్‌రౌండర్స్**: అబ్దుల్ సమద్, మార్కో యాన్సెన్, వాషింగ్టన్ సుందర్, సందీప్ శర్మ, నితీష్ కుమార్ రెడ్డి, విజయకుమార్ విశాక్
- **బౌలర్స్**: భువనేశ్వర్ కుమార్, టి. నటరాజన్, ఉమ్రాన్ మలిక్, ఫజల్హక్ ఫారూఖి, మయాంక్ మార్కండే

### సన్‌రైజర్స్ హైదరాబాద్ హోమ్ గ్రౌండ్

సన్‌రైజర్స్ హైదరాబాద్ హోమ్ గ్రౌండ్ రాజీవ్ గాంధీ ఇంటర్నేషనల్ క్రికెట్ స్టేడియం (ఉప్పల్ స్టేడియం). ఈ స్టేడియం 55,000 మంది ప్రేక్షకులను కలిగి ఉంటుంది మరియు హైదరాబాద్‌లోని ఉప్పల్‌లో ఉంది.
"""

# డాక్యుమెంట్‌ని చంక్స్‌గా విభజించడం
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_text(ipl_cricket_doc)

# ఎంబెడింగ్స్‌ని క్రియేట్ చేయడం
embeddings = OpenAIEmbeddings()

# వెక్టర్ స్టోర్‌ని క్రియేట్ చేయడం
vectorstore = FAISS.from_texts(chunks, embeddings)

# రిట్రీవర్‌ని క్రియేట్ చేయడం
retriever = vectorstore.as_retriever()

# ప్రాంప్ట్‌ని క్రియేట్ చేయడం
template = """
మీరు IPL క్రికెట్ గురించి నిపుణుడు. ఈ క్రింది కాంటెక్స్ట్ ఇన్ఫర్మేషన్‌ని ఉపయోగించి, ప్రశ్నకు సమాధానం ఇవ్వండి.
సమాధానం కాంటెక్స్ట్‌లో ఉన్న సమాచారానికి మాత్రమే పరిమితం చేయండి. మీకు తెలియని విషయాలు అడిగితే, "క్షమించండి, నాకు ఆ సమాచారం తెలియదు" అని చెప్పండి.

కాంటెక్స్ట్:
{context}

ప్రశ్న: {question}

సమాధానం:
"""

prompt = ChatPromptTemplate.from_template(template)

# LLM ని క్రియేట్ చేయడం
llm = ChatOpenAI(model="gpt-3.5-turbo")

# RAG చెయిన్‌ని క్రియేట్ చేయడం
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# RAG చెయిన్‌ని టెస్ట్ చేయడం
questions = [
    "IPL ఎప్పుడు ప్రారంభమైంది?",
    "సన్‌రైజర్స్ హైదరాబాద్ ఎన్ని సార్లు IPL ట్రోఫీని గెలుచుకుంది?",
    "సన్‌రైజర్స్ హైదరాబాద్ ప్రస్తుత కెప్టెన్ ఎవరు?",
    "IPL లో అత్యధిక ట్రోఫీలు గెలుచుకున్న జట్లు ఏవి?",
    "తెలుగు సినిమాలో ఎస్.ఎస్. రాజమౌళి గురించి చెప్పండి"
]

for question in questions:
    print(f"\nప్రశ్న: {question}")
    answer = rag_chain.invoke(question)
    print(f"సమాధానం: {answer}")

## మల్టిపుల్ డాక్యుమెంట్స్‌తో RAG

ఇప్పుడు, మనం రెండు డాక్యుమెంట్స్‌ని కలిపి ఒక RAG సిస్టమ్‌ని క్రియేట్ చేద్దాం.

In [ ]:
# రెండు డాక్యుమెంట్స్‌ని కలపడం
combined_docs = [telugu_cinema_doc, ipl_cricket_doc]

# డాక్యుమెంట్స్‌ని చంక్స్‌గా విభజించడం
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_chunks = []
for doc in combined_docs:
    chunks = text_splitter.split_text(doc)
    all_chunks.extend(chunks)

# ఎంబెడింగ్స్‌ని క్రియేట్ చేయడం
embeddings = OpenAIEmbeddings()

# వెక్టర్ స్టోర్‌ని క్రియేట్ చేయడం
vectorstore = FAISS.from_texts(all_chunks, embeddings)

# రిట్రీవర్‌ని క్రియేట్ చేయడం
retriever = vectorstore.as_retriever()

# ప్రాంప్ట్‌ని క్రియేట్ చేయడం
template = """
మీరు తెలుగు సినిమాలు మరియు IPL క్రికెట్ గురించి నిపుణుడు. ఈ క్రింది కాంటెక్స్ట్ ఇన్ఫర్మేషన్‌ని ఉపయోగించి, ప్రశ్నకు సమాధానం ఇవ్వండి.
సమాధానం కాంటెక్స్ట్‌లో ఉన్న సమాచారానికి మాత్రమే పరిమితం చేయండి. మీకు తెలియని విషయాలు అడిగితే, "క్షమించండి, నాకు ఆ సమాచారం తెలియదు" అని చెప్పండి.

కాంటెక్స్ట్:
{context}

ప్రశ్న: {question}

సమాధానం:
"""

prompt = ChatPromptTemplate.from_template(template)

# LLM ని క్రియేట్ చేయడం
llm = ChatOpenAI(model="gpt-3.5-turbo")

# RAG చెయిన్‌ని క్రియేట్ చేయడం
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# RAG చెయిన్‌ని టెస్ట్ చేయడం
questions = [
    "తెలుగు సినిమా చరిత్రలో మొదటి టాకీ ఏది?",
    "సన్‌రైజర్స్ హైదరాబాద్ ఎన్ని సార్లు IPL ట్రోఫీని గెలుచుకుంది?",
    "ఎస్.ఎస్. రాజమౌళి తెరకెక్కించిన ప్రముఖ చిత్రాలు ఏవి?",
    "IPL లో అత్యధిక ట్రోఫీలు గెలుచుకున్న జట్లు ఏవి?",
    "తెలుగు సినిమాలు మరియు IPL క్రికెట్ మధ్య ఏదైనా సంబంధం ఉందా?",
    "చంద్రబాబు నాయుడు గురించి చెప్పండి"
]

for question in questions:
    print(f"\nప్రశ్న: {question}")
    answer = rag_chain.invoke(question)
    print(f"సమాధానం: {answer}")

## RAG ని ఇంప్రూవ్ చేయడం

ఇప్పుడు, మనం RAG పెర్ఫార్మెన్స్‌ని ఇంప్రూవ్ చేయడానికి కొన్ని టెక్నిక్స్‌ని చూద్దాం.

In [ ]:
from langchain_core.runnables import RunnableParallel
from langchain_core.prompts import PromptTemplate

# క్వెరీ ట్రాన్స్‌ఫార్మేషన్
def transform_query(query):
    """క్వెరీని ట్రాన్స్‌ఫార్మ్ చేయడం"""
    llm = ChatOpenAI(model="gpt-3.5-turbo")
    prompt = ChatPromptTemplate.from_template(
        """మీరు సెర్చ్ క్వెరీ ఆప్టిమైజేషన్ నిపుణుడు. ఈ క్రింది యూజర్ క్వెరీని, వెక్టర్ డేటాబేస్‌లో సెర్చ్ చేయడానికి ఆప్టిమైజ్ చేయండి.
        
        యూజర్ క్వెరీ: {query}
        
        ఆప్టిమైజ్డ్ క్వెరీ:
        """
    )
    chain = prompt | llm | StrOutputParser()
    return chain.invoke({"query": query})

# రిట్రీవర్‌ని ఇంప్రూవ్ చేయడం
def get_relevant_documents(query):
    """ప్రశ్నకు సంబంధించిన డాక్యుమెంట్స్‌ని పొందడం"""
    transformed_query = transform_query(query)
    docs = retriever.get_relevant_documents(transformed_query)
    return docs

# ఇంప్రూవ్డ్ రిట్రీవర్‌ని క్రియేట్ చేయడం
improved_retriever = vectorstore.as_retriever(
    search_type="mmr",  # Maximum Marginal Relevance
    search_kwargs={"k": 5, "fetch_k": 10}
)

# ఇంప్రూవ్డ్ ప్రాంప్ట్‌ని క్రియేట్ చేయడం
improved_template = """
మీరు తెలుగు సినిమాలు మరియు IPL క్రికెట్ గురించి నిపుణుడు. ఈ క్రింది కాంటెక్స్ట్ ఇన్ఫర్మేషన్‌ని ఉపయోగించి, ప్రశ్నకు సమాధానం ఇవ్వండి.

సమాధానం ఇవ్వడానికి ఈ క్రింది స్టెప్స్‌ని ఫాలో అవ్వండి:
1. కాంటెక్స్ట్‌లో ఉన్న సమాచారాన్ని జాగ్రత్తగా చదవండి
2. ప్రశ్నకు సంబంధించిన సమాచారాన్ని గుర్తించండి
3. సమాధానాన్ని క్రియేట్ చేయండి, కాంటెక్స్ట్‌లో ఉన్న సమాచారానికి మాత్రమే పరిమితం చేయండి
4. మీకు తెలియని విషయాలు అడిగితే, "క్షమించండి, నాకు ఆ సమాచారం తెలియదు" అని చెప్పండి

కాంటెక్స్ట్:
{context}

ప్రశ్న: {question}

సమాధానం:
"""

improved_prompt = ChatPromptTemplate.from_template(improved_template)

# ఇంప్రూవ్డ్ RAG చెయిన్‌ని క్రియేట్ చేయడం
improved_rag_chain = (
    {"context": improved_retriever, "question": RunnablePassthrough()}
    | improved_prompt
    | llm
    | StrOutputParser()
)

# ఇంప్రూవ్డ్ RAG చెయిన్‌ని టెస్ట్ చేయడం
questions = [
    "తెలుగు సినిమా చరిత్రలో మొదటి టాకీ ఏది మరియు ఎప్పుడు విడుదలైంది?",
    "సన్‌రైజర్స్ హైదరాబాద్ IPL ట్రోఫీని ఎప్పుడు గెలుచుకుంది మరియు ఆ సమయంలో కెప్టెన్ ఎవరు?",
    "ఎస్.ఎస్. రాజమౌళి తెరకెక్కించిన ప్రముఖ చిత్రాలు ఏవి మరియు వాటిలో ఏది అంతర్జాతీయ గుర్తింపు పొందింది?",
    "IPL లో అత్యధిక ట్రోఫీలు గెలుచుకున్న జట్లు ఏవి మరియు ఎన్ని సార్లు గెలుచుకున్నాయి?",
    "తెలుగు సినిమాలు మరియు IPL క్రికెట్ మధ్య ఏదైనా సంబంధం ఉందా? ఉదాహరణలు ఇవ్వండి.",
    "చంద్రబాబు నాయుడు గురించి చెప్పండి"
]

for question in questions:
    print(f"\nప్రశ్న: {question}")
    answer = improved_rag_chain.invoke(question)
    print(f"సమాధానం: {answer}")

## ముగింపు

ఈ ట్యుటోరియల్‌లో, మనం LangChain లో RAG (Retrieval-Augmented Generation) గురించి నేర్చుకున్నాము:

1. RAG అంటే ఏమిటో మరియు దాని ప్రాముఖ్యత గురించి తెలుసుకున్నాము
2. RAG పైపైలైన్‌లోని వివిధ స్టెప్స్‌ని చూశాము
3. సింపుల్ RAG ఎగ్జాంపుల్‌ని చూశాము
4. మల్టిపుల్ డాక్యుమెంట్స్‌తో RAG ని ఇంప్లిమెంట్ చేయడం నేర్చుకున్నాము
5. RAG పెర్ఫార్మెన్స్‌ని ఇంప్రూవ్ చేయడానికి కొన్ని టెక్నిక్స్‌ని చూశాము

RAG అనేది LLM ని మన స్వంత డేటాతో ఎన్‌హాన్స్ చేయడానికి ఉపయోగపడే చాలా శక్తివంతమైన టెక్నిక్. ఇది LLM హాలుసినేషన్స్‌ని తగ్గించడానికి మరియు డొమైన్-స్పెసిఫిక్ నాలెడ్జ్‌ని అందించడానికి సహాయపడుతుంది.

తదుపరి ట్యుటోరియల్‌లో, మనం LangChain లో మరిన్ని అడ్వాన్స్డ్ కాన్సెప్ట్స్ గురించి నేర్చుకుంటాము.